In [1]:
%%capture
# @title 1. 라이브러리 설치
%pip install -q fastapi uvicorn[standard]
%pip install -q pyngrok>=7.0.0 diffusers>=0.27.0 transformers accelerate
%pip install -q peft controlnet_aux torch>=2.0.0 opencv-python-headless Pillow
%pip install -q requests safetensors pydantic huggingface_hub python-multipart
%pip install -U diffusers transformers accelerate safetensors
%pip install -q wget

print("✅ 라이브러리 설치 완료")

In [2]:
# @title InstantX 코드 파일 다운로드
import wget
import os

# Flux InstantX 파일 리스트
files = [
    ("pipeline_flux_ipa.py", "https://huggingface.co/InstantX/FLUX.1-dev-IP-Adapter/resolve/main/pipeline_flux_ipa.py?download=true"),
    ("transformer_flux.py", "https://huggingface.co/InstantX/FLUX.1-dev-IP-Adapter/resolve/main/transformer_flux.py?download=true"),
    ("attention_processor.py", "https://huggingface.co/InstantX/FLUX.1-dev-IP-Adapter/resolve/main/attention_processor.py?download=true"),
    ("infer_flux_ipa_siglip.py", "https://huggingface.co/InstantX/FLUX.1-dev-IP-Adapter/resolve/main/infer_flux_ipa_siglip.py?download=true"),
    ("ip-adapter.bin", "https://huggingface.co/InstantX/FLUX.1-dev-IP-Adapter/resolve/main/ip-adapter.bin?download=true"),
]

for filename, url in files:
    if not os.path.exists(filename):
        wget.download(url, out=filename)


In [3]:
# @title 2. Hugging Face 로그인 / Ngrok 설정 (Authtoken 시크릿 키)
import os
from google.colab import userdata
from huggingface_hub import login
from pyngrok import conf, ngrok

# Hugging Face 및 ngrok 토큰 (Colab Secret에서 가져오기)
hf_token = userdata.get("HF_TOKEN")        # Hugging Face Token
ngrok_token = userdata.get("NGROK_TOKEN")  # ngrok Token

if hf_token == None:
  print('x')
if ngrok_token == None:
  print('x')

# 포트 설정
PORT = 8000

# hugging face 로그인
print("🔑 Hugging Face 로그인 중...")
login(token=hf_token)

# ngrok 설정
if ngrok_token == "YOUR_NGROK_AUTHTOKEN":
  print("⚠️ 경고: Ngrok Authtoken을 입력하세요. 없으면 Ngrok 터널이 작동하지 않습니다.")
  print("Ngrok 토큰은 https://dashboard.ngrok.com/get-started/your-authtoken 에서 확인 가능합니다.")
else:
  os.environ['NGROK_AUTHTOKEN'] = ngrok_token
  conf.get_default().auth_token = ngrok_token
  print("✅ Ngrok Authtoken 설정 완료")

🔑 Hugging Face 로그인 중...
✅ Ngrok Authtoken 설정 완료


In [4]:
# @title 3. FastAPI 애플리케이션 코드 작성 (main_server.py 파일 생성)
# 이 셀은 FastAPI 서버 코드를 main_server.py 파일로 저장합니다.

%%writefile main_server.py
import os
import io
import logging
from contextlib import asynccontextmanager
from typing import Optional, Any

import torch
import uvicorn
from fastapi import FastAPI, HTTPException, UploadFile, File, Form
from fastapi.responses import Response
from PIL import Image
import gc

# IP-Adapter 관련 모듈
from pipeline_flux_ipa import FluxPipeline
from transformer_flux import FluxTransformer2DModel
from attention_processor import IPAFluxAttnProcessor2_0
from transformers import AutoProcessor, SiglipVisionModel
from infer_flux_ipa_siglip import resize_img, MLPProjModel, IPAdapter

# --- Configuration ---
BASE_MODEL_ID = "black-forest-labs/FLUX.1-dev"
IMAGE_ENCODER_PATH = "google/siglip-so400m-patch14-384"
IPADAPTER_PATH = "./ip-adapter.bin"

DEFAULT_STEPS = 30
# DEFAULT_GUIDANCE_SCALE = 3.5
DEFAULT_IPADAPTER_SCALE = 0.7  # 변경: IP-Adapter 용 Scale 이름
DEFAULT_LORA_SCALE = 0.8
IMAGE_WIDTH = 1024
IMAGE_HEIGHT = 1024
PORT = 8000

# --- Global State ---
pipe: Optional[FluxPipeline] = None
ip_model: Optional[IPAdapter] = None
device: Optional[str] = None

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- Helper Functions ---
def load_pil_image(image_bytes: bytes) -> Image.Image:
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    return image

def image_to_bytes(image: Image.Image) -> bytes:
    byte_arr = io.BytesIO()
    image.save(byte_arr, format='PNG')
    byte_arr.seek(0)
    return byte_arr.getvalue()

def get_generator(seed: Optional[int] = None) -> torch.Generator:
    if seed is None:
        seed = torch.randint(0, 2**32 - 1, (1,)).item()
    logger.info(f"Using seed: {seed}")
    return torch.Generator(device=device).manual_seed(seed)

async def prepare_reference_image(uploaded_image: UploadFile) -> Image.Image:
    image = load_pil_image(await uploaded_image.read())
    resized_image = resize_img(image)
    return resized_image

# --- Model Loading ---
def load_models():
    global pipe, ip_model, device

    device = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {device}")
    dtype = torch.bfloat16 if device == "cuda" and torch.cuda.is_bf16_supported() else torch.float16

    try:
        logger.info("Loading Flux Transformer...")
        transformer = FluxTransformer2DModel.from_pretrained(
            BASE_MODEL_ID, subfolder="transformer", torch_dtype=dtype
        )

        logger.info("Loading Flux Pipeline...")
        pipe = FluxPipeline.from_pretrained(
            BASE_MODEL_ID, transformer=transformer, torch_dtype=dtype
        )

        logger.info("Loading IP-Adapter...")
        ip_model = IPAdapter(
            pipe,
            IMAGE_ENCODER_PATH,
            IPADAPTER_PATH,
            device=device,
            num_tokens=128
        )

        pipe.to(device)
        pipe.enable_model_cpu_offload()
        pipe.enable_attention_slicing()

    except Exception as e:
        logger.exception("Fatal error during model loading")
        raise RuntimeError(f"Failed to load models: {e}")

    logger.info("✅ Model loading complete.")

# --- FastAPI Setup ---
@asynccontextmanager
async def lifespan(app: FastAPI):
    logger.info("Application startup...")
    load_models()
    yield
    logger.info("Application shutdown...")
    global pipe
    del pipe
    if device == 'cuda':
        torch.cuda.empty_cache()

app = FastAPI(lifespan=lifespan, title="Flux IP-Adapter Image API")

# --- API Endpoint ---
@app.post("/generate/ip-adapter-image", summary="Generate Image using IP-Adapter", response_class=Response)
async def generate_ip_adapter_image(
    prompt: str = Form(...),
    negative_prompt: Optional[str] = Form(""),  # Currently unused.
    lora_scale: Optional[float] = Form(DEFAULT_LORA_SCALE),
    ipadapter_scale: float = Form(DEFAULT_IPADAPTER_SCALE),
    num_inference_steps: int = Form(DEFAULT_STEPS),
    seed: Optional[int] = Form(None),
    image: UploadFile = File(...)
):
    if ip_model is None:
        raise HTTPException(status_code=503, detail="IP-Adapter model not ready.")

    reference_image = await prepare_reference_image(image)

    # Before Setting
    torch.cuda.empty_cache()
    gc.collect()
    pipe.to("cuda")

    generator = get_generator(seed)

    try:
        with torch.inference_mode():
            output_images = ip_model.generate(
                pil_image=reference_image,
                prompt=prompt,
                scale=ipadapter_scale,
                width=IMAGE_WIDTH,
                height=IMAGE_HEIGHT,
                seed=seed,
            )

        output_image = output_images[0]
        img_bytes = image_to_bytes(output_image)
        return Response(content=img_bytes, media_type="image/png")

    except Exception as e:
        logger.exception("Image generation with IP-Adapter failed")
        raise HTTPException(status_code=500, detail=f"Generation failed: {e}")

    finally:
        # After Setting (정상이든 에러든 무조건 실행)
        torch.cuda.empty_cache()
        gc.collect()
        pipe.to("cpu")


# --- Main ---
if __name__ == "__main__":
    logger.info("Starting Uvicorn server...")
    uvicorn.run("main_server:app", host="0.0.0.0", port=PORT, reload=False)


Writing main_server.py


In [5]:
# @title 4. ngrok 터널링 및 api 제공
# 환경 변수 설정
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

import time
import os
import subprocess
from pyngrok import ngrok
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
pyngrok_logger = logging.getLogger("pyngrok")
pyngrok_logger.setLevel(logging.INFO)

# --- 설정 ---
LOG_FILE = "uvicorn_server.log"
PORT = 8000  # main_server.py 내부 포트와 일치
STATIC_NGROK_DOMAIN = "publicly-capable-monkfish.ngrok-free.app"  # 고정 도메인

# --- 기존 서버 및 ngrok 프로세스 종료 ---
print("ℹ️ 기존 Uvicorn/Ngrok 프로세스 종료 시도...")

try:
    ngrok.kill()
    print("   - ngrok 프로세스 종료 완료 (pyngrok).")
    time.sleep(2)
except Exception as e:
    pyngrok_logger.warning(f"ngrok.kill() 실행 중 오류 발생 (무시 가능): {e}")

subprocess.run(['pkill', '-f', 'uvicorn main_server:app'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-f', f'ngrok.*http.*{PORT}'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-f', '/root/.config/ngrok/ngrok'], stderr=subprocess.DEVNULL)
time.sleep(3)

# --- FastAPI 서버 백그라운드 실행 ---
print(f"🚀 FastAPI 서버를 백그라운드에서 시작합니다... 로그 파일: {LOG_FILE}")
nohup_cmd = f"nohup python main_server.py > {LOG_FILE} 2>&1 &"
subprocess.Popen(nohup_cmd, shell=True)
time.sleep(5)  # 서버 초기 부팅 대기

# --- ngrok static 도메인으로 연결 ---
print(f"🌐 ngrok static domain 연결 시도 ({STATIC_NGROK_DOMAIN})...")
public_url = ngrok.connect(
    addr=PORT,
    proto="http",
    domain=STATIC_NGROK_DOMAIN
)
print(f"✅ 고정 URL 연결 완료: {public_url}")

# --- 서버 준비 대기 (모델 로딩 시간 확보) ---
wait_seconds = 200  # 필요에 따라 조정
print(f"⏳ 서버 및 모델 준비 대기 중 ({wait_seconds}초)...")
for i in range(wait_seconds):
    print(str(i), end=" ", flush=True)
    if (i + 1) % 30 == 0:
        print()
    time.sleep(1)
print("\n✅ 서버 준비 대기 완료.")

# --- 결과 출력 ---
print(f"\n🎯 서버가 백그라운드에서 실행되고 있으며 외부 접속 URL은 다음과 같습니다:")
print(f"🔗 {public_url}")

print("\n--- 실행 중인 관련 프로세스 (참고용) ---")
!ps -ef | grep -E "main_server.py|ngrok" | grep -v -E "grep|pkill|colab"


env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
ℹ️ 기존 Uvicorn/Ngrok 프로세스 종료 시도...
   - ngrok 프로세스 종료 완료 (pyngrok).
🚀 FastAPI 서버를 백그라운드에서 시작합니다... 로그 파일: uvicorn_server.log


INFO:pyngrok.ngrok:Opening tunnel named: http-8000-0f528c63-3f98-4fce-9532-47a1e19fdf2e


🌐 ngrok static domain 연결 시도 (publicly-capable-monkfish.ngrok-free.app)...


INFO:pyngrok.process:Overriding default auth token
INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:56+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:56+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:56+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:56+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:57+0000 lvl=info msg="client session established" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:57+0000 lvl=info msg="tunnel session started" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:57+0000 lvl=info msg=start pg=/api/tunnels id=5521723b951070b1
INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:57+0000 lvl=info msg=end pg=/api/tunnels id=5521723b9510

✅ 고정 URL 연결 완료: NgrokTunnel: "https://publicly-capable-monkfish.ngrok-free.app" -> "http://localhost:8000"
⏳ 서버 및 모델 준비 대기 중 (200초)...
0 

INFO:pyngrok.process.ngrok:t=2025-04-29T02:11:57+0000 lvl=info msg=end pg=/api/tunnels id=ea98bab78c31fc40 status=201 dur=42.515827ms


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 
90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 
120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 
150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 
180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 
✅ 서버 준비 대기 완료.

🎯 서버가 백그라운드에서 실행되고 있으며 외부 접속 URL은 다음과 같습니다:
🔗 NgrokTunnel: "https://publicly-capable-monkfish.ngrok-free.app" -> "http://localhost:8000"

--- 실행 중인 관련 프로세스 (참고용) ---
root        1263       1 74 02:11 ?        00:02:33 python3 main_server.py
root        1300     525  0 02:11 ?        00:00